In [1]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, f1_score # Using F1 score
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
import all_metrics as all_metrics # Assuming your custom metrics function is available
import credit_card_fraud_utils as ccf



In [2]:
config = ccf.Config()
# Load the dataset
df = ccf.load_data(config)

# Display basic information
print(f"Dataset Shape: {df.shape}")
print(f"Number of Transactions: {df.shape[0]}")
print(f"Number of Features: {df.shape[1]}")

# Display the first few rows
df.head()

Dataset Shape: (284807, 31)
Number of Transactions: 284807
Number of Features: 31


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Class'])
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)



### Random search

In [4]:


# This is the recommended way to handle imbalance directly within XGBoost
neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
# Handle potential division by zero if pos_count is 0 (though unlikely if it's the minority class)
scale_pos_weight_value = 1.0 if pos_count == 0 else neg_count / pos_count
print(f"Target scale_pos_weight for XGBoost: {scale_pos_weight_value:.2f}")
# ---

# 1. Define Pipeline with Scaler and XGBoost Classifier
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()), # Keep scaler for consistency
    ('resampler', None),          # Placeholder for resampling step
    ('classifier', XGBClassifier(
        random_state=42,
        objective='binary:logistic', # Objective for binary classification
        eval_metric='logloss',       # Common evaluation metric during training
        use_label_encoder=False      # Suppress warning in newer XGBoost versions
    ))
])

# 2. Define Expanded Parameter Grid for XGBoost

# Common XGBoost parameter ranges to explore
xgb_param_ranges = {
    'classifier__n_estimators': [100, 200, 300, 400, 500], # More trees?
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__max_depth': [3, 5, 7, 9], # Deeper trees?
    'classifier__subsample': [0.7, 0.8, 0.9, 1.0], # Row sampling
    'classifier__colsample_bytree': [0.7, 0.8, 0.9, 1.0], # Column sampling per tree
    'classifier__gamma': [0, 0.1, 0.5, 1, 2], # Min loss reduction for split
    'classifier__reg_alpha': [0, 0.01, 0.1, 1], # L1 regularization
    'classifier__reg_lambda': [0.1, 1, 5, 10] # L2 regularization (often start > 0)
}

# Expanded resampler parameters
smote_k_neighbors = [3, 5, 10]
adasyn_n_neighbors = [3, 5, 10]
rus_sampling_strategy = [0.2, 0.35, 0.5] # Ratio minority/majority after RUS
ros_sampling_strategy = [0.5, 0.7, 1.0] # Ratio minority/majority after ROS

# List to hold all grid dictionaries
param_grid_xgb = []

# --- Configuration for NO Resampling ---

param_grid_xgb.append({
    'resampler': [None],
    **xgb_param_ranges,
    'classifier__scale_pos_weight': [1, scale_pos_weight_value] # Test default vs calculated weight
})

# --- Configurations for Resampling Methods --
resampling_options_list = [
    {'resampler': [SMOTE(random_state=42)], 'resampler__k_neighbors': smote_k_neighbors},
    {'resampler': [ADASYN(random_state=42)], 'resampler__n_neighbors': adasyn_n_neighbors},
    {'resampler': [RandomOverSampler(random_state=42)], 'resampler__sampling_strategy': ros_sampling_strategy},
    {'resampler': [RandomUnderSampler(random_state=42)], 'resampler__sampling_strategy': rus_sampling_strategy},
    {'resampler': [TomekLinks(sampling_strategy='auto')]},
    {'resampler': [NearMiss(version=1)]}
]

for resampler_config in resampling_options_list:
    config = {
        **resampler_config,
        **xgb_param_ranges,
        'classifier__scale_pos_weight': [1] # Set to 1 (no weight) when resampling
    }
    param_grid_xgb.append(config)

# 3. Configure RandomizedSearchCV
n_iterations_xgb = 75 # Adjust based on time/resources (e.g., 50-150)
scoring_metric = 'f1' # Target F1 score directly
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_search_xgb = RandomizedSearchCV(
    pipeline_xgb,
    param_distributions=param_grid_xgb,
    n_iter=n_iterations_xgb,
    scoring=scoring_metric,
    refit=scoring_metric, # Refit the best model based on F1 score
    cv=cv_strategy,
    verbose=2,            # Show detailed progress
    n_jobs=-1,            # Use all CPU cores
    random_state=42
)

# 4. Run the Search
print(f"\nStarting RandomizedSearchCV for XGBoost with n_iter={n_iterations_xgb}...")
print(f"Optimizing for '{scoring_metric}' score.")
print(f"Best Logistic Regression Test F1 was: 0.6951") # Remind ourselves of the target



Target scale_pos_weight for XGBoost: 577.29

Starting RandomizedSearchCV for XGBoost with n_iter=75...
Optimizing for 'f1' score.
Best Logistic Regression Test F1 was: 0.6951


In [5]:

random_search_xgb.fit(X_train, y_train)

# 5. Process Results (After Fitting)
print("\nBest XGBoost Parameters Found:")
print(random_search_xgb.best_params_)
print("\nBest CV " + scoring_metric.upper() + " Score:")
print(random_search_xgb.best_score_)

Fitting 5 folds for each of 75 candidates, totalling 375 fits


/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:19:48] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:19:48] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:19:48] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/tom/anaconda3/envs/is

[CV] END classifier__colsample_bytree=0.9, classifier__gamma=0.5, classifier__learning_rate=0.2, classifier__max_depth=7, classifier__n_estimators=100, classifier__reg_alpha=0.1, classifier__reg_lambda=10, classifier__scale_pos_weight=577.2868020304569, classifier__subsample=0.7, resampler=None; total time=   3.4s
[CV] END classifier__colsample_bytree=0.9, classifier__gamma=0.5, classifier__learning_rate=0.2, classifier__max_depth=7, classifier__n_estimators=100, classifier__reg_alpha=0.1, classifier__reg_lambda=10, classifier__scale_pos_weight=577.2868020304569, classifier__subsample=0.7, resampler=None; total time=   3.4s


/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/Users/tom/anacon

[CV] END classifier__colsample_bytree=0.9, classifier__gamma=0.1, classifier__learning_rate=0.2, classifier__max_depth=7, classifier__n_estimators=300, classifier__reg_alpha=1, classifier__reg_lambda=0.1, classifier__scale_pos_weight=577.2868020304569, classifier__subsample=0.9, resampler=None; total time=   4.6s
[CV] END classifier__colsample_bytree=0.9, classifier__gamma=0.1, classifier__learning_rate=0.2, classifier__max_depth=7, classifier__n_estimators=300, classifier__reg_alpha=1, classifier__reg_lambda=0.1, classifier__scale_pos_weight=577.2868020304569, classifier__subsample=0.9, resampler=None; total time=   4.9s


...

/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/Users/tom/anacon

[CV] END classifier__colsample_bytree=0.8, classifier__gamma=1, classifier__learning_rate=0.2, classifier__max_depth=5, classifier__n_estimators=500, classifier__reg_alpha=0, classifier__reg_lambda=1, classifier__scale_pos_weight=1, classifier__subsample=0.8, resampler=ADASYN(random_state=42), resampler__n_neighbors=3; total time=  11.8s
[CV] END classifier__colsample_bytree=0.7, classifier__gamma=0, classifier__learning_rate=0.2, classifier__max_depth=9, classifier__n_estimators=400, classifier__reg_alpha=0.01, classifier__reg_lambda=0.1, classifier__scale_pos_weight=1, classifier__subsample=1.0, resampler=ADASYN(random_state=42), resampler__n_neighbors=10; total time=  16.0s


/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tom/anaconda3/envs/islp/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/Users/tom/anacon

[CV] END classifier__colsample_bytree=0.7, classifier__gamma=0, classifier__learning_rate=0.2, classifier__max_depth=9, classifier__n_estimators=400, classifier__reg_alpha=0.01, classifier__reg_lambda=0.1, classifier__scale_pos_weight=1, classifier__subsample=1.0, resampler=ADASYN(random_state=42), resampler__n_neighbors=10; total time=  15.6s
[CV] END classifier__colsample_bytree=0.8, classifier__gamma=1, classifier__learning_rate=0.2, classifier__max_depth=5, classifier__n_estimators=500, classifier__reg_alpha=0, classifier__reg_lambda=1, classifier__scale_pos_weight=1, classifier__subsample=0.8, resampler=ADASYN(random_state=42), resampler__n_neighbors=3; total time=  11.9s


KeyboardInterrupt: 

In [ ]:
y_pred_if = best_if_model.predict(X_test)

# Convert predictions from (-1, 1) to (0, 1) format
y_pred_if = np.where(y_pred_if == -1, 1, 0)

accuracy_if = accuracy_score(y_test, y_pred_if)
precision_if = precision_score(y_test, y_pred_if, average='binary')
recall_if = recall_score(y_test, y_pred_if, average='binary')
f1_if = f1_score(y_test, y_pred_if, average='binary')


print(f"Accuracy: {accuracy_if:.4f}")
print(f"Precision: {precision_if:.4f}")
print(f"Recall: {recall_if:.4f}")
print(f"F1-score: {f1_if:.4f}")
precision_if, recall_if, _ = precision_recall_curve(y_test, y_pred_if)
auprc_if = auc(recall_if, precision_if)
print(f"AUPRC: {auprc_if:.4f}")

Accuracy: 0.9974
Precision: 0.2547
Recall: 0.2755
F1-score: 0.2647
AUPRC: 0.2657


In [ ]:

print("\n--- Evaluating Best XGBoost Model on Test Set ---")
best_pipeline_xgb = random_search_xgb.best_estimator_
results_xgb = all_metrics.calculate_model_metrics(
    best_pipeline_xgb, X_test, y_test, 'XGBoost (RandomizedSearch Best)'
)
# Compare the key metric
print(f"\nComparison of Test F1-Scores:")
print(f"  Best XGBoost Test F1: {results_xgb.get('F1-score', 'N/A'):.4f}")
print(f"  Best Logistic Regression Test F1: 0.6951")

### Refined Grid search

In [ ]:
model_xgb = XGBClassifier()
model_xgb.fit(Xsmote, ysmote)
y_pred_xgb = model_xgb.predict(X_test)

xgb_results = all_metrics.calculate_model_metrics(model_xgb, X_test, y_test, 'XGBoost')